In [13]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\WJA-
[nltk_data]     Laptop\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\WJA-
[nltk_data]     Laptop\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# Load the data
true_data = pd.read_csv("../data/True.csv")
false_data = pd.read_csv("../data/Fake.csv")

true_data['label'] = 1
false_data['label'] = 0
data = pd.concat([true_data, false_data], ignore_index=True)

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(text)
    words = [word for word in words if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if not word in stop_words]
    return ' '.join(words)

data['title'] = data['title'].apply(preprocess_text)
data['text'] = data['text'].apply(preprocess_text)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['title'] + ' ' + data['text'])
sequences = tokenizer.texts_to_sequences(data['title'] + ' ' + data['text'])
max_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, data['label'], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [15]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.2))  # Add dropout for regularization
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Add batch normalization after Conv1D and LSTM layers
model.add(BatchNormalization())

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with the augmented data
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=128)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/10
225/225 [==============================] - 52s 220ms/step - loss: 0.6685 - accuracy: 0.8729 - val_loss: 0.2735 - val_accuracy: 0.9523
Epoch 2/10
225/225 [==============================] - 49s 220ms/step - loss: 0.0982 - accuracy: 0.9798 - val_loss: 0.1388 - val_accuracy: 0.9648
Epoch 3/10
225/225 [==============================] - 50s 220ms/step - loss: 0.0567 - accuracy: 0.9902 - val_loss: 0.3114 - val_accuracy: 0.9038
Epoch 4/10
225/225 [==============================] - 52s 233ms/step - loss: 0.0412 - accuracy: 0.9938 - val_loss: 0.1423 - val_accuracy: 0.9720
Epoch 5/10
225/225 [==============================] - 51s 226ms/step - loss: 0.0228 - accuracy: 0.9983 - val_loss: 0.1358 - val_accuracy: 0.9736
Epoch 6/10
225/225 [==============================] - 50s 220ms/step - loss: 0.0200 - accuracy: 0.9985 - val_loss: 0.1749 - val_accuracy: 0.9733
Epoch 7/10
225/225 [==============================] - 49s 220ms/step - loss: 0.0183 - accuracy: 0.9987 - val_loss: 0.1668 - val_ac